In [3]:
import re
import matplotlib.pyplot as plt
import jieba
import numpy as np
from collections import Counter,defaultdict
import pandas as pd
import tqdm
import pkuseg 
from sklearn.model_selection import train_test_split

In [6]:
train_file = "data/train.csv"
test_file = "data/test_new.csv"
sample_file= "data/sample.csv"

In [7]:
df_train=pd.read_csv(train_file, delimiter="\t")
df_test=pd.read_csv(test_file, delimiter=",")

X_train_raw=np.array(df_train["comment"])
y_train_raw=np.array(df_train["label"])
X_test_raw=np.array(df_test["comment"])
X_test_id=np.array(df_test["id"])

# pat=re.compile(r'[\u4e00-\u9fa5]+')
# X_train_raw=[ "".join(pat.findall(x)) for x in X_train_raw]
# X_test_raw=[ "".join(pat.findall(x)) for x in X_test_raw]

In [8]:
def process_file(X_train_raw,X_test_raw):
    """
    读取训练数据和测试数据，并对它们做一些预处理
    """
    seg=pkuseg.pkuseg()
    
    X_train=[" ".join(seg.cut(x)) for x in X_train_raw]
    X_test=[" ".join(seg.cut(x)) for x in X_test_raw]
    
    return X_train,X_test
    

In [4]:
import numpy as np

In [7]:
a=np.load('mydict.npy',allow_pickle=True)

In [9]:
X_train_beforesplit,X_test=process_file(X_train_raw,X_test_raw)

In [10]:
len(X_train_beforesplit)

10000

In [11]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train_beforesplit,y_train_raw,test_size=0.3,stratify=y_train_raw,random_state=0)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import datasets
import sklearn
from sklearn import svm
from sklearn.utils import shuffle

## tfidf

In [13]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")  
X_train_tfidf = vectorizer.fit_transform(X_train)
word = vectorizer.get_feature_names() 
X_valid_tfidf = vectorizer.transform(X_valid)  # 测试数据的特征
print (np.shape(X_train_tfidf), np.shape(y_train),np.shape(X_valid_tfidf),np.shape(y_valid))

X_train1,y_train1=shuffle(X_train_tfidf,y_train)

(7000, 8664) (7000,) (3000, 8664) (3000,)


In [14]:
model=svm.SVC(C=10**0.5958581168005016)
print(model.get_params)
model.fit(X_train1,y_train1)
print(model.score(X_train1,y_train1))
y_pred=model.predict(X_valid_tfidf)
sklearn.metrics.f1_score(y_pred,y_valid,average='macro')

<bound method BaseEstimator.get_params of SVC(C=3.943284546565891, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)>
1.0


0.914611618234793

> test:0.88

## tfidf+bigram

In [11]:
train_comments_beforetfidf=[]
valid_comments_beforetfidf=[]
test_comments_beforetfidf=[]

for index,value  in enumerate(X_train):
    lentext=len(value.split())
    value=value.split()
    
    for i in range(lentext-1):
        value.append("".join(value[i:i+2]))
    train_comments_beforetfidf.append(" ".join(value))

print(len(train_comments_beforetfidf))


for index,value  in enumerate(X_valid):
    lentext=len(value.split())
    value=value.split()
    
    for i in range(lentext-1):
        value.append("".join(value[i:i+2]))
    valid_comments_beforetfidf.append(" ".join(value))

print(len(valid_comments_beforetfidf))

for index,value  in enumerate(X_test):
    lentext=len(value.split())
    value=value.split()
    
    for i in range(lentext-1):
        value.append("".join(value[i:i+2]))
    test_comments_beforetfidf.append(" ".join(value))

print(len(test_comments_beforetfidf))

7000
3000
2000


In [12]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")  
X_train_tfidf = vectorizer.fit_transform(train_comments_beforetfidf)
word = vectorizer.get_feature_names() 
X_valid_tfidf = vectorizer.transform(valid_comments_beforetfidf)  # 测试数据的特征
print (np.shape(X_train_tfidf), np.shape(y_train),np.shape(X_valid_tfidf),np.shape(y_valid))

X_train1,y_train1=shuffle(X_train_tfidf,y_train)

(7000, 63389) (7000,) (3000, 63389) (3000,)


In [14]:
model=svm.SVC(C=1.2020614011792285,class_weight="balanced")
print(model.get_params)
model.fit(X_train1,y_train1)
print(model.score(X_train1,y_train1))
y_pred=model.predict(X_valid_tfidf)
sklearn.metrics.f1_score(y_pred,y_valid,average='macro')

<bound method BaseEstimator.get_params of SVC(C=1.2020614011792286, break_ties=False, cache_size=200,
    class_weight='balanced', coef0=0.0, decision_function_shape='ovr', degree=3,
    gamma='scale', kernel='rbf', max_iter=-1, probability=False,
    random_state=None, shrinking=True, tol=0.001, verbose=False)>
1.0


0.9138185757904067

> test:0.87

In [73]:
# from bayes_opt import BayesianOptimization
# from sklearn.model_selection import cross_val_score

# def svc_cv(C, data, targets):
#     estimator = svm.SVC(C=C, random_state=2,class_weight="balancf1_score 
#     cval = cross_val_score(estimator, data, targets, scoring='f1_macro', cv=4)
#     return cval.mean()

# def optimize_svc(data, targets):

#     def svc_crossval(expC):
#         C =  expC
#         return svc_cv(C=C, data=data, targets=targets)

#     optimizer = BayesianOptimization(
#         f=svc_crossval,
#         pbounds={"expC": (1.1, 7)},
#         random_state=12,
#         verbose=2
#     )
#     optimizer.maximize(init_points=3,n_iter=10)

#     print("Final result:", optimizer.max)
    
# print("--- Optimizing SVM ---")
# optimize_svc(X_train1,y_train1)

--- Optimizing SVM ---
|   iter    |  target   |   expC    |
-------------------------------------
|  1        |  0.8913   |  2.01     |
|  2        |  0.8902   |  5.466    |
|  3        |  0.8902   |  2.654    |
|  4        |  0.8902   |  6.998    |
|  5        |  0.8983   |  1.102    |
|  6        |  0.8983   |  1.1      |
|  7        |  0.8983   |  1.1      |
|  8        |  0.8983   |  1.1      |
|  9        |  0.8983   |  1.1      |
|  10       |  0.8983   |  1.1      |
|  11       |  0.8983   |  1.1      |
|  12       |  0.8983   |  1.1      |
|  13       |  0.8983   |  1.1      |
Final result: {'target': 0.8982854109763347, 'params': {'expC': 1.1020614011792285}}


## 数据保存

In [17]:
X_test_tfidf = vectorizer.transform(test_comments_beforetfidf)

In [18]:
y_pred=model.predict(X_test_tfidf)
df_sample=pd.read_csv(sample_file, delimiter=",")
df_sample["label"]=y_pred
df_sample.to_csv("answer.csv",index=False)

In [19]:
df_look=pd.read_csv("answer.csv", delimiter=",")
df_look

,id,label
0,0011f384-9e54-4fb4-a272-330a6cab6804,0
1,00223e4f-47e1-4fc8-9657-06444a7de9a5,0
2,00225350-c169-435c-84cf-970068df5b12,0
3,00a3190c-90c1-44c3-b809-7a9b1314cd27,0
4,00b3f76e-fda3-42cd-8884-25e03a5dba64,0
...,...,...
1995,ffbd3c14-56ba-412f-ac68-9f4ccbafe4f5,0
1996,ffcc4330-2b02-485b-a3bb-e1c7d42baaae,1
1997,ffcc53f6-7cf9-4e5e-9294-a9a33c7568f8,0
1998,ffd3ca4b-dc5f-4dbd-b249-bac065045870,0
